# Dataset Analysis
In this Jupyter Notebook, we analyse dataset from Airbus Ship Detection Challenge.
Main task in this challenge it's "find ships on satellite images as quickly as possible"

In [ ]:
import os

# Finding how many images we have
num_train_images = len(os.listdir('dataset/train_v2'))
num_test_images = len(os.listdir('dataset/test_v2'))
print(f'Number of images for training: {num_train_images}')
print(f'Number of images for testing: {num_test_images}')

In [ ]:
# Make function for viewing a random image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_type):
    # Setup target directory
    target_folder = target_dir+target_type

    # Get a random image path
    random_image = random.sample(os.listdir(target_folder), 1)

    # Read the image and plot it
    img = mpimg.imread(target_folder + "\\" + random_image[0])
    plt.imshow(img)
    plt.title(random_image)
    plt.axis('off')

    # show the shape of the image
    print(f"Imgae shape: {img.shape}")

    # returning image
    return img

In [ ]:
# View a random image from the training dataset
img = view_random_image(target_dir='dataset\\',
                        target_type='train_v2')

In [ ]:
# View the tensor of image
img

In [ ]:
# View the image shape
img.shape  # returns ( width, height, colour channels)

In [ ]:
# Get all the pixel values between 0 & 1 (normalization)
img/255.

In [61]:
import pandas as pd

# # Check the train_ship_segmentations_v2.csv file to use it later
# load train_ship_segmentations_v2.csv to pandas DataFrame
train_ground_truth = pd.read_csv("dataset\\train_ship_segmentations_v2.csv")
train_ground_truth.head()
num_masks = train_ground_truth.shape[0]
num_masks

(231723, 2)

In [ ]:
train_ground_truth.ImageId[0]

In [ ]:
train_ground_truth.ImageId[2]

In [ ]:
train_ground_truth.EncodedPixels[2]

In [57]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

# create function for decoding run-length mask from "train_ship_segmentations_v2.csv"
def tf_rle_decode(rle_string, shape=(256, 256)):
    """
    Function for decoding run-length encoding mask from string.

    :param rle_string: run-length string from csv file
    :param shape: shape of output image
    :return: numpy array as image mask
    """
    shape = tf.convert_to_tensor(shape, tf.int64)
    size = tf.math.reduce_prod(shape)

    decoded = []
    encoded = []
    rle_tensor = tf.strings.split(rle_string)
    rle_tensor = tf.strings. to_number(rle_tensor, tf.int64)

    starts = rle_tensor[::2] - 1
    lens = rle_tensor[1::2]

    # Make ones to be scattered
    total_ones = tf.reduce_sum(lens)
    ones = tf.ones([total_ones], tf.uint8)

    # Make scattering indices
    rle_range = tf.range(total_ones)
    lens_cum = tf.math.cumsum(lens)
    rle_ssorted = tf.searchsorted(lens_cum, rle_range, 'right')
    idx = rle_range + tf.gather(starts - tf.pad(lens_cum[:-1], [(1, 0)]), rle_ssorted)

    # Scatter ones into flattened mask
    mask_flat = tf.scatter_nd(tf.expand_dims(idx, 1), ones, [size])

    # Reshape into mask
    return tf.reshape(mask_flat, shape)


    # for i in range(len(rle_array)):
    #     if i == 0: continue
    #     encoded.append((rle_array[i-1], rle_array[i]))
    #
    # encoded = np.array(encoded)
    # for rl in encoded:
    #     r, p = rl[0], rl[1]
    #     decoded.extend([p]*r)
    #
    # dimg = np.array(decoded).reshape(shape)
    # return dimg

In [58]:
dimg = tf_rle_decode(train_ground_truth.EncodedPixels[2], [256, 256])
plt.show(dimg)

InvalidArgumentError: indices[0] = [264660] does not index into shape [65536] [Op:ScatterNd]